In [ ]:
import requests
import pandas as pd
import numpy as np
import io
import os
from datetime import date
import datetime
import glob
import matplotlib.pyplot as plt
import shutil
import re
plt.ioff()

In [ ]:
def save_to_csv(df: pd.DataFrame, path, name, index = False):
    """
    Saving dataframe to csv file
    df: dataframe
    path: destination path
    name: name of the file
    """
    if not os.path.exists(path):
        os.makedirs(path)
    if path[-1] == '/':
        df.to_csv(path+name, index = index)
    else:
        df.to_csv(path + '/' + name, index = index)
    return True

In [ ]:
def download(url, path: str = None, headers = {'content-type': 'application/json', 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'}):
    """
    Dowloading files from the give URL using requests module, with specified header and store to desire path if provided. 
    Otherwises, store to the default path that is under the current directory called with region specified
    url: url to download files
    path: path to store files
    headers: headers that use for the request.get()
    """
    
    response2 = requests.get(url, headers=headers)
    # print(response2.status_code)
    if response2.status_code != 200:
        raise Exception(f'Code error, {response2.status_code}')
    df = pd.read_csv(io.StringIO(response2.text))
    
    last_bash = url.rfind('/')
    name =  url[last_bash + 1 :]

    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        save_to_csv(df, path, name)
    except:
        print('出错了')
    return path

In [ ]:
def one_hour_interval(src:str, dst:str, name:str):
    """
    Using the combined csv to reduce to 1 hour interval
    src: source
    dst: Destination
    """
    df = pd.read_csv(src)
    df['SETTLEMENTDATE'] =  pd.to_datetime(df['SETTLEMENTDATE'])
    df = df.set_index(df['SETTLEMENTDATE'])
    new_df = df.groupby(pd.Grouper(freq='60min')).mean()
    save_to_csv(new_df, dst, name, True)

In [ ]:
def find_all_files(path = '', name = '*', end = '.csv') -> list:
    if path[-1] != '/':
        path += '/'
    result = []
    result += glob.glob(path + name + end)
    return result

In [ ]:
def combine_csv(src:list, dst, name):
    """
    Combine .csv files to the specific dir with name total_REG.csv
    Three different .csv files are generated, original file, dropped negative number file and nagative to zero file
    src: source path
    dst: destination path
    name: name
    """
    
    # src = src[:-1]
    all_files = []
    for i in range(len(src)):
        all_files.extend(find_all_files(path = src[i] , name = '*'))
    li = []
    
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame2 = frame.loc[frame['RRP'] > 0]
    frame3 = frame.copy()
    frame3.loc[(frame3.RRP < 0),'RRP']= 0

    if dst[-1] != '/':
         dst += '/'

    if not os.path.exists(dst):
        os.makedirs(dst)
        
    name1 = 'total.csv'
    name2 = 'total_reduced.csv'
    name3 = 'total_toZero.csv'
    save_to_csv(frame, dst, name1)
    save_to_csv(frame2, dst, name2)
    save_to_csv(frame3, dst, name3)

In [ ]:
def check_exist(path):
    """
    check if path exists, True if exists else False
    path: desire path
    """
    if path[-1] != '/':
        path += '/'
    return os.path.exists(path)

In [ ]:
def listdirs(rootdir):
    paths = []
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            paths.append(d)
            listdirs(d)
    return paths

In [ ]:
def plot_data(year:list = ['22', '21', '19', '18', '17'], reg:list = ['SA', 'QLD', 'VIC', 'TAS', 'NSW'], csv_names:list = ['total_reduced.csv', 'total_toZero.csv', 'total.csv'], path:str = '', amount_path:str = 'Splited.csv'):
    """
    plotting data

    reg: regions
    csv_names: name of the csv names
    path: path to save the graph
    amount_path: path to the amount csv data
    """
    amount = pd.read_csv(amount_path)
    Total_year_dirs = listdirs('Total_year_one_hour')
    if path != '':
        path += '/'
    for dir in Total_year_dirs:
        for j in year:
            if j in dir:
                for k in reg:
                    if k in dir:
                        for i in csv_names:
                            plot_yearly_graph(dir+'/'+i, amount, path + 'Graphs/' + i[:-4] + dir[dir.find('/'):])

In [ ]:
def delete_wd(path:str):
    if path[-1] != '/':
        path += '/'
    shutil.rmtree(path)
    return True

In [ ]:
def bar_plot(x1:list, x_name:list, y_name:str, title:str, path:str, width = 0.95):
    """
    Plot the bar graph with the given data and parameters

    x1: data
    x_name: name of the x-axis
    y_name: name of the y-axis
    title
    path: path where to store the graph
    width: width of the bar, deault value is 0.95
    """
    if not check_exist(path):
        os.makedirs(path)
    if path[-1] != '/':
        path += '/'
    ind = np.arange(len(x1))  # the x locations for the groups
    width = width       # the width of the bars
    fig = plt.figure()

    fig.set_figwidth(80)
    fig.set_figheight(50)

    ax = fig.add_subplot(111)
    colormat=np.where(x1>0, 'b','r')
    rects1 = ax.bar(ind, x1, width, color=colormat, edgecolor = 'white')
    # rects2 = ax.bar(ind+width, x2, width, color='seagreen')

    # add some
    ax.set_ylabel(y_name, fontsize = 40)
    ax.set_title(title, fontsize = 40)
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(x_name, fontsize = 40)
    plt.xticks(rotation=90)
    xlocs, xlabs = plt.xticks()
    for i, v in enumerate(x1): 
        if v > 0:
            plt.text(xlocs[i] - 1, v + 50, str(round(v, 2)), fontsize = 30)
        else:
            plt.text(xlocs[i] - 1, 0 , str(round(v, 2)), fontsize = 30)
    # plt.show()
    plt.savefig(path + title + '.jpg')
    

In [ ]:
def annuel_report(price:pd.DataFrame, amount:pd.DataFrame, name:str):
    print('ann: ', 'Graphs/Annuel Summary/' + name[name.find('/')+ 1:name.rfind('/')+ 1 ])
    # name = name[name.rfind('/') + 1 :]
    index = 0
    df = pd.DataFrame()
    df_index = []
    while index + 24 <= len(price):
        data = price.iloc[index:index + 24, 2] * amount.iloc[index:index + 24, 1]
        data.reset_index(drop=True, inplace=True)
        df = pd.concat([df, data], axis = 1)
        index += 24
        df_index.append(price.iloc[index, 0][:10])
    df = df.set_axis(df_index, axis=1, inplace=False)
    df['Avg'] = df.mean(axis = 1)
    

    save_to_csv(df, 'Graphs/Annuel Summary/'+name[name.find('/')+ 1:name.rfind('/')+ 1] , name[name.rfind('/') + 1:]+'.csv')

In [ ]:
def plot_yearly_graph(total_year_chart:str, amount:pd.DataFrame, save_path:str):
    """
    Plot the data yearly. Automatically detect if the data has been calculated and plotted
    
    total_year_chart: path that point to the prices csv
    amount: simulation of the electricity that is generated by the solar system
    save_path: path that stores the the graph
    """
    price = pd.read_csv(total_year_chart)
    
    curruent_day = -1
    if check_exist(save_path):
        if '.DS_Store' in os.listdir(save_path):
            curruent_day = (len(os.listdir(save_path)) - 1) * 24
        else:
            curruent_day = len(os.listdir(save_path)) * 24
    if curruent_day > 0:
        index = curruent_day
    elif curruent_day == (len(price)-1)/24:
        return 
    else:
        index = 0

    annuel_report(price, amount, save_path)
    
    while index + 24 <= len(price):
        os.system('clear')
        if check_exist(save_path + '/Day ' + str((index+24)/24) + '.jpg'):
            index += 24
        else:
            x_axis = price.iloc[index:index + 24, 0]
            data = price.iloc[index:index + 24, 2] * amount.iloc[index:index + 24, 1]
            print(str(datetime.datetime.strptime(price.iloc[index - 1, 0], '%Y-%m-%d %H:%M:%S').date()))
            bar_plot(data, x_axis, 'Earning', str(datetime.datetime.strptime(price.iloc[index - 1, 0], '%Y-%m-%d %H:%M:%S').date()), save_path)
            index += 24

In [ ]:
if __name__ == '__main__':
    urls, paths = {'SA1': [], 'QLD1' : [], 'VIC1':[], 'TAS1':[], 'NSW1':[]}, {'SA1': [], 'QLD1' : [], 'VIC1':[], 'TAS1':[], 'NSW1':[]}
    regions = ['NSW1', 'SA1', 'VIC1', 'QLD1', 'TAS1']
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


    for state in regions:
        for j in range(int(str(date.today().year)[-2:]), int(str(date.today().year)[-2:]) - 6, -1):
            if int(str(date.today().year)[-2:]) == j:
                for i in range(1, int(date.today().month)):
                    if i < 10:
                        url = 'https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_20' + str(j) + '0' + str(i) + '_' + state + '.csv'
                    else:
                        url = 'https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_20' + str(j) + str(i) + '_' + state + '.csv'
                    
                    bucket = []
                    bucket.append(url)
                    bucket.append([state[:-1], str(j)])
                    urls[state].append(bucket)  
            else:
                for i in range(1,13):
                    if i < 10:
                        url = 'https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_20' + str(j) + '0' + str(i) + '_' + state + '.csv'
                    else:
                        url = 'https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_20' + str(j) + str(i) + '_' + state + '.csv'
                    
                    bucket = []
                    bucket.append(url)
                    bucket.append([state[:-1], str(j)])
                    urls[state].append(bucket)

    # for i in regions:
    #     if check_exist(i):
    #         raise Exception(f'The path exists, please delete it follow this dir: {os.getcwd()}' + '/' + f'{i}')

    flag = bool()
    for reg in regions:
        print(reg)
        flag = check_exist(reg[:-1])
        if flag:
            break
    # print(flag)
    if flag:
        con = input('some data has already existed, [Y] for cover the previous data, [N] use the current data.')
        if con.lower() == 'y':
            for reg in regions:
                if check_exist(reg):
                    delete_wd(reg)
            
            # delete_wd('Total/')

            for reg in regions:
                for i in urls[reg]:
                    temp = download(i[0], '/'.join(i[1]))
                    if temp not in paths[reg]:
                        paths[reg].append(temp)
                
                # combine_csv(paths[reg], 'Total/', reg[:-1])
        else:
            if os.getcwd()+'/Total' in listdirs(os.getcwd()):
                con2 = input('The combined CSV is found, do you want to delete them and create new csv? [Y]es [N]o')
                if con2.lower() == 'y':
                    delete_wd('Total/')
            for reg in regions:
                paths[reg].extend(listdirs(reg[:-1]))
                
    else:
        for reg in regions:
                for i in urls[reg]:
                    temp = download(i[0], '/'.join(i[1]))
                    if temp not in paths[reg]:
                        paths[reg].append(temp)
                
                # combine_csv(paths[reg], 'Total/', reg[:-1])


In [ ]:
    if not check_exist('Total_year'):
        for i in paths.values():
            for j in i:
                combine_csv([j], 'Total_year/'+str(j.replace('/', '-')), j.replace('/','_'))

In [ ]:
    # Reduce to 1 hour interval
    if not check_exist("Total_year_one_hour"):
        before_ave_files = []
        Total_year_dirs = listdirs('Total_year')
        for i in Total_year_dirs:
            before_ave_files.extend(find_all_files(path = i))
        for i in before_ave_files:
            one_hour_interval(i, 'Total_year_one_hour' + i[i.find('/') : i.rfind('/')], i[i.rfind('/'):])

# 图表

In [17]:
    desire_reg = input('地区(SA, QLD, TAS, VIC, NSW)，无输入为处理全部: ').upper()
    type_file = input('模式(total_reduced.csv, total_toZero.csv, total.csv), 无输入为处理全部：')
    amount_path = input('模拟发电量的地址, 无输入为处理全部：')
    path = input('储存地址，无输入为储存在程序所在地址: ')
    year = input('年份（只输入最后两位数），无输入为处理全部：')
    year = re.split(' |, |,', year)
    if amount_path == '':
        plot_data(year, re.split(' |, |,', desire_reg), re.split(' |, |,', type_file), path)
    else:
        plot_data(year, re.split(' |, |,', desire_reg), re.split(' |, |,', type_file), path, amount_path)

2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25
2020-12-26
2020-12-27
2020-12-28
2020-12-29
2020-12-30
